In [ ]:

song = input('Please enter the song name: ')
song = song.lower()
songs_file = pd.read_csv (r"C:\Users\Barbara\OneDrive\Documents\GitHub\IHLabs_BV\Week6\songs_data.csv")

def lowercase(songs_file):
    songs_file = songs_file.applymap(lambda s:s.lower() if type(s) == str else s)
    return songs_file


def random_rec(songs_file):
    song_choice = songs_file.sample()
    val_song = song_choice["title"].item()
    val_artist = song_choice["artist"].item()
    return val_song, val_artist

lower_df = lowercase(songs_file)

if song in lower_df.values :
    print("your song is in the top 100")
    title, artist  = random_rec(songs_file)
    print("I have a recommandation for you:   " + title + " by " + artist)
else:
    print("Try again")

![logo_ironhack_blue 7](https://user-images.githubusercontent.com/23629340/40541063-a07a0a8a-601a-11e8-91b5-2f13e4e6b441.png)

# Lab | Web Scraping Multiple Pages

#### Business goal:

- Check the `case_study_gnod.md` file.
- Make sure you've understood the big picture of your project:

  - the goal of the company (`Gnod`),
  - their current product (`Gnoosic`),
  - their strategy, and
  - how your project fits into this context.

  Re-read the business case and the e-mail from the CTO, take a look at the flowchart and create an initial Trello board with the tasks you think you'll have to accomplish.

#### Instructions 

#### Prioritize the MVP

In the previous lab, you had to scrape data about "hot songs". It's critical to be on track with that part, as it was part of the request from the CTO.

If you couldn't finish the first lab, use this time to go back there.

**User experience:**

- What happens if the user inputs a song that doesn't exist?
- What do we do with songs that have the same name, but a different artist?
- How do we deal with typos?

**Architecture:**

- Do we build the interaction with the user in the same notebook as the web-scraping?
- Where do we store the scraped songs?

**Scheduling / Automation:**

- Should we scrape billboard / wikipedia every time a user sends a request?

**Testing:**

- Does it work when you test it with a real user (a colleague)?

Chances are that more issues will appear, and that not all of them will be solved during this session. But what's important is that the issues have been identified.

# Optional
#### Expand the project

If you're done, you can try to expand the project on your own. Here are a few suggestions:

- Find other lists of hot songs on the internet and scrape them too: having a bigger pool of songs will be awesome!
- Apply the same logic to other "groups" of songs: the best songs from a decade or from a country / culture / language / genre.
- Wikipedia maintains a large collection of lists of songs: https://en.wikipedia.org/wiki/Lists_of_songs

#### Practice web scraping

As you've seen, scraping the internet is a skill that can get you all sorts of information. Here are some little challenges that you can try to gain more experience in the field:

- Retrieve an arbitrary Wikipedia page of "Python" and create a list of links on that page: `url ='https://en.wikipedia.org/wiki/Python'`
- Find the number of titles that have changed in the United States Code since its last release point: `url = 'http://uscode.house.gov/download/download.shtml'`
- Create a Python list with the top ten FBI's Most Wanted names: `url = 'https://www.fbi.gov/wanted/topten'`
- Display the 20 latest earthquakes info (date, time, latitude, longitude and region name) by the EMSC as a pandas dataframe: `url = 'https://www.emsc-csem.org/Earthquake/'`
- List all language names and number of related articles in the order they appear in [wikipedia.org](wikipedia.org): `url = 'https://www.wikipedia.org/'`
- A list with the different kind of datasets available in [data.gov.uk](data.gov.uk): `url = 'https://data.gov.uk/'`
- Display the top 10 languages by number of native speakers stored in a pandas dataframe: `url = 'https://en.wikipedia.org/wiki/List_of_languages_by_number_of_native_speakers'`




# Case Study: The site for recommendations - "Gnod"

### Scenario

You have been hired as a Data Analyst for "Gnod".

"Gnod" is a site that provides recommendations for music, art, literature and products based on collaborative filtering algorithms. Their flagship product is the _music recommender_, which you can try at [www.gnoosic.com](https://www.gnoosic.com). The site asks users to input 3 bands they like, and computes similarity scores with the rest of the users. Then, they recommend to the user bands that users with similar tastes have picked.

"Gnod" is a small company, and its only revenue stream so far are Ads in the site. In the future, they would like to explore partnership options with music apps (such as _Deezer_, _Soundcloud_ or even _Apple Music_ and _Spotify_). However, for that to be possible, they need to expand and improve their recommendations.

That's precisely where you come. They have hired you as a Data Analyst, and they expect you to bring a mix of technical expertise and business mindset to the table.

Jane, CTO of Gnod, has sent you an email assigning you with your first task.

### Task(s)

> This is an e-mail Jane - CTO of Gnod - sent over your inbox in the first weeks working there.

_Dear xxxxxxxx,
We are thrilled to welcome you as a Data Analyst for *Gnoosic*!_

_As you know, we are trying to come up with ways to enhance our music recommendations. One of the new features we'd like to research is to recommend songs (not only bands). We're also aware of the limitations of our collaborative filtering algorithms, and would like to give users two new possibilities when searching for recommendations:_

- _Songs that are actually similar to the ones they picked from an acoustic point of view._
- _Songs that are popular around the world right now, independently from their tastes._

_Coming up with the perfect song recommender will take us months - no need to stress out too much. In this first week, we want you to explore new data sources for songs. The Internet is full of information and our first step is to acquire it do an initial exploration. Feel free to use APIs or directly scrape the web to collect as much information as possible from popular songs. Eventually, we'll need to collect data from millions of songs, but we can start with a few hundreds or thousands from each source and see if the collected features are useful._

_Once the data is collected, we want you to create clusters of songs that are similar to each other. The idea is that if a user inputs a song from one group, we'll prioritize giving them recommendations of songs from that same group._

_On Friday, you will present your work to me and Marek, the CEO and founder. Full disclosure: I need you to be very convincing about this whole song-recommender, as this has been my personal push and the main reason we hired you for!_

_Be open minded about this process: we are agile, and that means that we define our products and features on-the-go, while exploring the tools and the data that's available to us. We'd love you to provide your own vision of the product and the next steps to be taken._

_Lots of luck and strength for this first week with us!_

_-Jane_


## User Experience
What happens if the user inputs a song that doesn't exist?

In [1]:
from csv import DictReader
import pandas as pd
import random

In [25]:
def lowercase(songs_file):
    songs_file = songs_file.applymap(lambda s:s.lower() if type(s) == str else s)
    return songs_file


def random_rec(songs_file,song):
    songs_file.drop(songs_file.loc[songs_file['title']== song].index, inplace =True)
    song_choice = songs_file.sample()
    val_song = song_choice["title"].item()
    val_artist = song_choice["artist"].item()

    return val_song, val_artist

In [28]:
#Collecting The Input As A Variable:#
song = input('Please enter the song name: ')
song = song.lower()
songs_file = pd.read_csv (r"C:\Users\Barbara\OneDrive\Documents\GitHub\IHLabs_BV\Week6\songs_data.csv")

songs_file = lowercase(songs_file)

if song in songs_file.values :
    print("Your song is at top 100, yeahh")
    title, artist  = random_rec(songs_file, song)
    print("I have a recommandation for you !!!:   " + title + " by " + artist)
else:
    print("Try again, the song is not at the Top 100, sorry")

Please enter the song name: as it was
Your song is at top 100, yeahh
I have a recommandation for you !!!:   yet to come by bts


In [27]:
songs_file

,title,artist
0,jimmy cooks,drake featuring 21 savage
1,as it was,harry styles
2,first class,jack harlow
3,wait for u,future featuring drake & tems
4,about damn time,lizzo
...,...,...
95,n95,kendrick lamar
96,love me more,sam smith
97,new truck,dylan scott
98,she's all i wanna be,tate mcrae
